<div class="alert alert-block alert-info">
<b>Multi-Head attention </b> 
<p>    
Multi Head attention is something but calculating multiple Causal Attention with different weight matrix for QUERY , KEY and VALUE 
and concatenating then together.

For anology consider the statement <font color=blue> "Atul vists Mumbai in Winters" </font>.

For this statement or input sequence to create multi attentions meaning we are trying to find answers for :

<span style=color:blue>
    
What is happening ?? => Visit

Who is visiting ?? => Atul

WHen is Atul visisting => Winter

</span>

So we create heads (causal attention) for each of these questions and then stack/concatenate them as context rich input for pre-training

</p>
</div>

---

In [4]:
import torch
import torch.nn as nn

<div class="alert alert-block alert-info">
<b>Multi-Head attention Class</b> 
<p>    

Lets create the Mutli-head attention Wrapper.

This wrapper calls the Causual attention for N number of time.

This N is number of attention heads we want to create
</p>
</div>

In [5]:
#Image this is the Token embedding for the input sentence = "your joruney starts with one step"

inputs = torch.tensor(
 [[0.43, 0.15, 0.89],  # Your
 [0.55, 0.87, 0.66],  # journey
 [0.57, 0.85, 0.64],  # starts
 [0.22, 0.58, 0.33],  # with
 [0.77, 0.25, 0.10],  # one
 [0.05, 0.80, 0.55]] # step
)

In [6]:
# We will have dataloader that will build the datasets in batches
# we create a batch with 3 input , for simiplicity now
# meaning we will have 3 sentence , with 6 tokens each , and 3 dimension , in batch pf 3

batch=torch.stack((inputs,inputs,inputs),dim=0)

In [7]:
class CausalAttention(nn.Module):
    def __init__(self, d_int , d_out, context_length, dropout, qkv_bias=False):
        super().__init__()

        # Step 1 :- Define the query , key and valye matrices with randowm values
        self.W_query = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_key = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_value = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)  # Define the dropout
        self.register_buffer('upper_mask_matrix' , torch.triu(torch.ones(context_length , context_length) , diagonal=1))
        

    def forward(self, x):

        # STep2 : create the key , queries and values by multiplying token embedding
        batch , num_tokens , test = x.shape
        keys    = self.W_key(x)
        queries = self.W_query(x)
        values  = self.W_value(x)

        # Step 3 : Create attention scores , these are raw attention score
        # The transpose(1,2) means just consider the data (tokens & dimensions) , as input has dimension = [ batch , token , dimension]
        attention_scores_raw = queries @ keys.transpose(1,2)

        # Step 4 :-  Implement the Causal Attention mechanism , buy masking future
        #            Input sequences after the given token
        attention_scores_raw.masked_fill_(
            self.upper_mask_matrix.bool()[:num_tokens , :num_tokens], -torch.inf) # [:num_tokens ,:num_tokens] is list comprehension .
                                                                            # If we have less tokens in one batch (E.g End of datasets with just 2 words) , to ensure it works we specify the dim  so it is broadcasted

        # Step 5 :- Normalize the attention scores by diding then with sqrt od dims and softmax
        attention_scores_norm = torch.softmax(
              attention_scores_raw / keys.shape[-1]**0.5 , dim=-1
            )


        # Step 6: Add the droput funtionality  . it is regularization method to prevent overfitting    
        attention_scores = self.dropout(attention_scores_norm)

        
        # Step 7: Create context vector
        context_vector = attention_scores @ values
        return context_vector

<div class="alert alert-block alert-info">
<b>Torch nn ModuleList</b> 
<p>    

This is very simple.

In the __init__ method of the class , we have intialized the "heads" with Module=CausalAttention with its paramters.

This is defined as a list with items=num_head (in this case it is 2).

Then this List if called or iterated in forward method with input=x
</p>
</div>

In [14]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in , d_out , context_length , dropout , num_heads , qkv_bias=False):
        super().__init__()
        print("calling Init")
        self.heads = nn.ModuleList(
                                    [CausalAttention(d_in , d_out , context_length ,dropout ,qkv_bias)
                                     for _ in range(num_heads)]
                                    )


    def forward(self , x):
       print("calling this")
       return torch.cat([head(x) for head in self.heads] , dim=-1)

In [22]:
torch.manual_seed(123)
context_length = batch.shape[1]
d_in=batch.shape[2]
d_out=2

ca = MultiHeadAttentionWrapper(d_in , d_out , context_length , 0.0 ,2)
context_vector = ca(batch)

calling Init
calling this


In [23]:
print(context_vector)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)


<div class="alert alert-block alert-info">
<b>Multi-Head attention Context Vector Dimension</b> 
<p>    
There is very imporatnt dimension changes to be noticed in the context vector.

We are concatenating the contect vectors from each heads along the columns (dim=1) . 
So that for each head with d_out parameter we get d_out * num_heads columns added.

Say for example , in above example we have set d_out =2 (this is the dimensions of Weight matrices QUERY , KEY and VALUE)  . 
We have defined the num of heads (num_heads=2) .
Each Causal attentioon will give us 2 dimension vector for 2 time hecne 2*2 = 4.


If this num_head was set to 5 (num_heads=2 ) then d_out * num_heads (2*5=10)
</p>
</div>

<div class="alert alert-block alert-info">
<b>Multi-Head attention </b> 
<p>   
<span>
The method shown above is not efficient.
We are using sequential method to perform this.

We will use different method tom perform this
</span>
</p>
</div>